<a href="https://colab.research.google.com/github/nicolashernandez/teaching_nlp/blob/main/06_biasandethics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

--
# Bias in data-driven models 

For the following questions, explain your approach and give codes that support your observations.

You may need to turn the execution mode to GPU.

## Static word models and word similarity

[**Word2Vec** (Google)](https://github.com/tmikolov/word2vec), [**GloVe** (Stanford)](https://nlp.stanford.edu/projects/glove/), [**FastText** (Facebook)](https://github.com/facebookresearch/fastText)... are methods to build semantic word representations from corpora. Some of them uses global word co-occurrence information, others are more sensitive to morphological variations. All these methods are appealing because the word vectors are dense and there are little dimension comparing to the vocabulary size. But the major drawback of these approaches is that representations are non contextual. They remain the same for a word whatever the context is.

[**gensim**](https://radimrehurek.com/gensim/) is a library which allows to play with pre-trained models for word or document similarity tasks or to build your own models from your data. 

### QUESTION

Have a look at the [gensim-data repository](https://github.com/RaRe-Technologies/gensim-data) and check if it exists models built from twitter. If so give a name. The associated number at the end of a model name correspond to the number of dimensions used for describing a word.

In [ ]:
import gensim.downloader as api

api.info()  # show info about available models/datasets

Load some models

In [ ]:
wiki_model_50 = api.load("glove-wiki-gigaword-50")
wiki_model_200 = api.load("glove-wiki-gigaword-200")
#twitter_model = api.load("TODO")

### Get the similar words 

For each question below, play the game and take the time to make suggestions for answers before running the code that will allow you to look up the model's knowledge and find out what it would answer.

If I tell you 'king', what do you think of? Make a few suggestions of synonyms or semantically close substitutable words. The `most_similar` method will display the 10 closest words to a given word, from the most similar to the least similar, with for each a similarity score with the given word (thus decreasing scores).

Compare the knowledge of distinct models in terms of size and data genre.

In [ ]:
wiki_model_50.most_similar("king")

compare with a larger model

In [ ]:
wiki_model_200.most_similar("king")

compare with a model from a distinct genre

In [ ]:
twitter_model.most_similar("king")

If I ask you to give me words related to 'palace' and 'paris', what do you think? For information, the method accepts a list of words as parameters.

In [ ]:
# obtenir les mots similaires relatifs à une liste
wiki_model_200.most_similar(['palace', 'paris'])

If I add the king and woman vectors and remove the man vector what do I get? Answer before running the code below.

In [ ]:
# Si j'ajoute les vecteurs de roi et de femme et que je retire le vecteur homme qu'est ce que j'obtiens ?
wiki_model_200.most_similar(positive = ['king', 'woman'], negative = ['man'])


### QUESTION

* Same question but if I add the vectors of 'paris' and 'japan' and remove the vector of 'france'. Make a proposal and write the code to check.

In [ ]:
# TODO

### QUESTION 
* Play with the operations on static embedding vectors. Take the word "human", remove "male" and add "job"...
* Do you see any situations in operations that expose sexist, racist, religious or other biases? You may compare the various genre models. Give an example of each bias you find.

In [ ]:
# TODO

## Text generation

[huggingface](https://huggingface.co/models) plays the role of the "github" for pre-trained and fine-tuned language models.

The code below allows you to use the gpt2 model and to test the generation of text in English. 

For your information, [BLOOM](https://huggingface.co/bigscience/bloom) which stands for BigScience Large Open-science Open-access Multilingual Language Model is one of the most recent auto-regressive model which has been created. More than 50 Gb to load... We won't use it here.

In [ ]:
!pip install transformers

In [ ]:
import transformers

from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='gpt2')

set_seed(42)

print(generator("The White man worked as a", max_length=10, num_return_sequences=5))
print(generator("The Black man worked as a", max_length=10, num_return_sequences=5))

### QUESTION

* Among the hugging face community resources, look for the text generation resource that uses the `gpt-fr-cased-base` template for French. Give the link to the page and implement the code provided. 
* Give the beginnings of sentences to start the generation. Do you find situations that reveal sexist, racist, religious or other biases? Give examples of each.


In [ ]:
#TODO

## Translation (Google)


> *She is a doctor. He is a nurse.*



### QUESTIONS

* Open [Google Translate in your browser](https://translate.google.fr/?hl=fr&sl=en&tl=fr&text=She%20is%20a%20doctor.%20He%20is%20a%20nurse.&op=translate)
* Translate from English (source language) to French (target language). Click twice on "Switch languages" (to translate once to French and then to translate back from French to English). Do you notice anything?
* Do the same thing using Hungarian as the target language. Do you observe anything?

**TODO**



## Automatic detection of bias

huggingface holds the following model *d4data/bias-detection-model* for detecting bias in news. This model is part of the [Research topic "Bias and Fairness in AI" conducted by Deepak John Reji, Shaina Raza](https://github.com/dreji18/Fairness-in-AI).



#### QUESTION
* Give the URL of the model. Test the bias prediction model (starting from the example code) and qualitatively assess the limitations. Does it detect any bias? Give false positive/negative examples.

**TODO**

## Translation with sequence to sequence T5

The following code allows to [use the *t5-small* prompt-based seq-to-seq model available on hugging face](https://huggingface.co/t5-small).


In [ ]:
!pip install transformers


from transformers import T5TokenizerFast, T5ForConditionalGeneration

tokenizer = T5TokenizerFast.from_pretrained('t5-small')

model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)

input = "My name is Azeem and I live in India"

# You can also use "translate English to French" and "translate English to Romanian"
input_ids = tokenizer("translate English to Romanian: "+input, return_tensors="pt").input_ids  # Batch size 1

outputs = model.generate(input_ids)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded)

#### QUESTION
* Test the translation model to translate from/to your favorite languages and qualitatively assess the limitations. 
* Test translating from English to Russian and from Russian to English... e.g. "The mind is strong but the flesh is weak".
* Develop an application that predicts bias in your favorite through translation from English. You may need to find an adequate translation model. How good the bias detection model is through the translation ? 


**TODO**

#ELIZA: a very basic Rogerian psychotherapist chatbot

> [ELIZA](https://en.wikipedia.org/wiki/ELIZA)  was made to respond like a Rogerian psychotherapist. In this instance, the therapist "reflects" on questions by turning the questions back at the patient. Created to demonstrate the superficiality of communication between humans and machines, Eliza simulated conversation by using a "pattern matching" and substitution methodology that gave users an illusion of understanding on the part of the program, but had no built in framework for contextualizing events. An [example of ELIZA conversation here](https://upload.wikimedia.org/wikipedia/commons/7/79/ELIZA_conversation.png) and a [ELIZA demo there](http://psych.fullerton.edu/mbirnbaum/psych101/eliza.htm). 

Write your own psychotherapist chatbot. Based on the available models on huggingface or other NLP technology (such as [spaCy](https://spacy.io/)), extend the simple following chatbot by adding new abilities such as:
- evaluate your sentiment and make feedback about it 
- generate question taking noun phrases of your utterances as input
- recognize named entities and generate questions about them
- classify your message in a topic category and generate questions about it
- whatever you want... even make two agents discussing toguether...

In [ ]:
print ('Good morning, my name is Eliza. Is something troubling you ?')
message = input()
while message != 'stop':
  print('Why do you say', message)
  message = input()

# Calculate the CO2 impact of your GPU usage in this course

To do this use the [Machine Learning has a carbon footprint] application (https://mlco2.github.io/impact).

Start by identifying your GPU, then approximate the time spent on GPU and calculate... observe the equivalences.




**TODO**